In [1]:
import torch
from torch import nn
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torch import optim
import time
import util_fei

In [3]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    if not torch.is_grad_enabled():
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2,4)
        if len(X.shape) == 2:
            # 这里是计算特征维度的均值，而并不是自己理解的批量的维度
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 计算通道维度的均值，同时将形状保持不变
            mean = X.mean(dim=(0,2,3), keepdim=True)
            var = ((X-mean) ** 2).mean(dim=(0,2,3), keepdim=True)
        X_hat = (X - mean) / torch.sqrt(var + eps)

        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    
    Y = gamma * X_hat + beta

    return Y, moving_mean.data, moving_var.data

In [4]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))

        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)

        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean, self.moving_var,
            eps=1e-5, momentum=0.9)

        return Y

In [19]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform= transforms.Compose([
                    # transforms.Resize(224),
                    transforms.ToTensor()])
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=transforms.Compose([
        # transforms.Resize(224),
        transforms.ToTensor()])
)

batch_size = 256

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [7]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), BatchNorm(6, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), BatchNorm(16, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(16*4*4, 120), BatchNorm(120, num_dims=2), nn.Sigmoid(),
    nn.Linear(120, 84), BatchNorm(84, num_dims=2), nn.Sigmoid(),
    nn.Linear(84, 10))

In [17]:
class LenNet(nn.Module):
    def __init__(self):
        super(LenNet, self).__init__()
        self.backbone_net = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            BatchNorm(6, num_dims=4),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            BatchNorm(16, num_dims=4),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(16*4*4,120),
            BatchNorm(120, num_dims=2),
            nn.Sigmoid(),
            nn.Linear(120,84),
            BatchNorm(84, num_dims=2),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

    def forward(self, X):
        logits = self.backbone_net(X)
        return logits

model = LenNet()
# print(model)

X = torch.randn(size=(1, 1, 28, 28))

for blk in model.backbone_net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Conv2d output shape:	 torch.Size([1, 6, 24, 24])
BatchNorm output shape:	 torch.Size([1, 6, 24, 24])
Sigmoid output shape:	 torch.Size([1, 6, 24, 24])
AvgPool2d output shape:	 torch.Size([1, 6, 12, 12])
Conv2d output shape:	 torch.Size([1, 16, 8, 8])
BatchNorm output shape:	 torch.Size([1, 16, 8, 8])
Sigmoid output shape:	 torch.Size([1, 16, 8, 8])
AvgPool2d output shape:	 torch.Size([1, 16, 4, 4])
Flatten output shape:	 torch.Size([1, 256])
Linear output shape:	 torch.Size([1, 120])
BatchNorm output shape:	 torch.Size([1, 120])
Sigmoid output shape:	 torch.Size([1, 120])
Linear output shape:	 torch.Size([1, 84])
BatchNorm output shape:	 torch.Size([1, 84])
Sigmoid output shape:	 torch.Size([1, 84])
Linear output shape:	 torch.Size([1, 10])


In [18]:
model = LenNet().to(device)
creterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1)

epoch_nums = 10

util_fei.train_model(model, train_dataloader, test_dataloader,creterion, optimizer,device, epoch_num=epoch_nums)

training on  cuda:0
epoch: 1,	                train_loss: 0.736100,	                train_correct:0.731583,	                test_correct:0.536500,	                test time:10.977388
epoch: 2,	                train_loss: 0.473468,	                train_correct:0.825517,	                test_correct:0.695600,	                test time:9.044344
epoch: 3,	                train_loss: 0.405161,	                train_correct:0.851300,	                test_correct:0.793000,	                test time:9.014734
epoch: 4,	                train_loss: 0.361974,	                train_correct:0.866467,	                test_correct:0.779600,	                test time:8.998656
epoch: 5,	                train_loss: 0.333811,	                train_correct:0.877133,	                test_correct:0.770100,	                test time:9.049407
epoch: 6,	                train_loss: 0.313243,	                train_correct:0.884183,	                test_correct:0.777500,	                test time:10.352089
epoch:

In [32]:
#使用框架的
class LenNet2(nn.Module):
    def __init__(self):
        super(LenNet2, self).__init__()
        self.backbone_net = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(16*4*4,120),
            nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

    def forward(self, X):
        logits = self.backbone_net(X)
        return logits

model = LenNet2()
# print(model)

X = torch.randn(size=(2, 1, 28, 28))

for blk in model.backbone_net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Conv2d output shape:	 torch.Size([2, 6, 24, 24])
BatchNorm2d output shape:	 torch.Size([2, 6, 24, 24])
Sigmoid output shape:	 torch.Size([2, 6, 24, 24])
AvgPool2d output shape:	 torch.Size([2, 6, 12, 12])
Conv2d output shape:	 torch.Size([2, 16, 8, 8])
BatchNorm2d output shape:	 torch.Size([2, 16, 8, 8])
Sigmoid output shape:	 torch.Size([2, 16, 8, 8])
AvgPool2d output shape:	 torch.Size([2, 16, 4, 4])
Flatten output shape:	 torch.Size([2, 256])
Linear output shape:	 torch.Size([2, 120])
BatchNorm1d output shape:	 torch.Size([2, 120])
Sigmoid output shape:	 torch.Size([2, 120])
Linear output shape:	 torch.Size([2, 84])
BatchNorm1d output shape:	 torch.Size([2, 84])
Sigmoid output shape:	 torch.Size([2, 84])
Linear output shape:	 torch.Size([2, 10])


In [ ]:
model = LenNet2().to(device)
creterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1)

epoch_nums = 10

util_fei.train_model(model, train_dataloader, test_dataloader,creterion, optimizer,device, epoch_num=epoch_nums)